In [40]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pylab as plt
import numpy as np
from PIL import Image

import os 
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [41]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"


In [42]:
# import images
#change user
path = "C:\\Users\\mkingsl6\\Desktop\\ViTPose-main\\LiftingData_cnn"
deadlift = path + "\\Deadlift"
bench = path + "\\Bench"
squats = path + "\\Squat"

In [47]:
# Convert deadlift to jpg
pos_paths_list = [deadlift, bench, squats]

filenames= []
for pos_paths in pos_paths_list:
    files = []
    for file in os.listdir(pos_paths):
        # check the files which are end with specific extension
        if file.endswith(".png"):
            # print path name of selected files
            file_path = os.path.join(pos_paths, file)
            # print(file_path)
            im = Image.open(file_path)
            new_name = file.split(".")[0] + ".jpg"
            # filenames.append(new_name)
            # files.append(new_name)
            # im.save(new_name)
            Image.open(file_path).convert('RGB').save(pos_paths+"\\"+new_name)


    for file in os.listdir(pos_paths):
        # check the files which are end with specific extension
        if file.endswith(".jpeg"):
            # print path name of selected files
            file_path = os.path.join(pos_paths, file)
            # print(file_path)
            im = Image.open(file_path)
            new_name = file.split(".")[0] + ".jpg"
            
            
            # filenames.append(new_name)
            # im.save(new_name)
            Image.open(file_path).convert('RGB').save(pos_paths+"\\"+new_name)
    for file in os.listdir(pos_paths):
        files.append(new_name)
    filenames.append(files)

In [44]:
# # normalise the pixel values
# train_ = ImageDataGenerator(validation_split = .2, rescale=1./255)
# TRAIN_DIR = path
# train_dataset = train_.flow_from_directory(
#     TRAIN_DIR,
#     target_size = (256, 256),
#     subset='training',
#     batch_size = 10,
#     class_mode='categorical'
# )

# validation_dataset = train_.flow_from_directory(
#     TRAIN_DIR,
#     target_size = (256, 256),
#     subset='validation',
#     class_mode='categorical',
#     batch_size = 10,
# )

In [48]:
def parse_function(filename, label):
    """Function that returns a tuple of normalized image array and labels array.
    Args:
        filename: string representing path to image
        label: 0/1 one-dimensional array of size N_LABELS
    """
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [256, 256])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    return image_normalized, label

In [49]:
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically to reduce GPU and CPU idle time


# Create a first dataset of file paths and labels
dataset = tf.data.Dataset.from_tensor_slices((filenames, ['squat','bench','deadlift']))
# Parse and preprocess observations in parallel
dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    

# This is a small dataset, only load it once, and keep it in memory.
dataset = dataset.cache()
    # Shuffle the data each buffer size
dataset = dataset.shuffle(buffer_size=1024)
        
# Batch the data for multiple steps
dataset = dataset.batch(256)
# Fetch batches in the background while the model is training.
dataset = dataset.prefetch(buffer_size=AUTOTUNE)

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [ ]:
# train_dataset.shape

In [ ]:
feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(256,256,3))
feature_extractor_layer.trainable = False


In [ ]:
neural_net = tf.keras.Sequential([
  feature_extractor_layer,
  # tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(1024, activation='relu', name='hidden_layer'),
  tf.keras.layers.Dense(3, activation='sigmoid', name='output')
])

ValueError: Exception encountered when calling layer "keras_layer_2" (type KerasLayer).

in user code:

    File "c:\Users\mkingsl6\.conda\envs\cnn_notebook\lib\site-packages\tensorflow_hub\keras_layer.py", line 237, in call  *
        result = smart_cond.smart_cond(training,

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (4 total):
        * <tf.Tensor 'inputs:0' shape=(None, 256, 256, 3) dtype=float32>
        * False
        * False
        * 0.99
      Keyword arguments: {}
    
     Expected these arguments to match one of the following 4 option(s):
    
    Option 1:
      Positional arguments (4 total):
        * TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='inputs')
        * False
        * False
        * TensorSpec(shape=(), dtype=tf.float32, name='batch_norm_momentum')
      Keyword arguments: {}
    
    Option 2:
      Positional arguments (4 total):
        * TensorSpec(shape=

In [ ]:
neural_net.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 3)                 3843      
                                                                 
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
neural_net.compile(
  model_optimizer=tf.keras.optimizers.Adam(),
  model_loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  model_metrics=['acc'])

TypeError: Invalid keyword argument(s) in `compile()`: ({'model_metrics', 'model_loss', 'model_optimizer'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".

In [ ]:
model_fit = neural_net.fit(train_dataset, epochs=6, validation_data=validation_dataset)      


In [ ]:
test_score = neural_net.evaluate(validation_dataset)